Notebook for harry testing

In [39]:
import sqlite3
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib.pyplot as plt

# Import the data from the sqlite file (locally for now)
# Matches table is stored in df
db_path = r"C:\Users\Harry Hall\OneDrive - UNSW\Desktop\MASTERS\S1\Research & Innovation\laliga.sqlite"
conn = sqlite3.connect(db_path)
query = "SELECT * FROM Matches"
df = pd.read_sql_query(query, conn)
conn.close()

Now need to clean the data and process into usable features.  
  
Firslty, dropping rows with no score and then splitting score into home and away component.  
  
I think it also makes sense to use the final notation suggested in the assignment of: home_win = 1, away_win = 2, tie = X  
  

In [40]:
df = df.dropna(axis = 'index' , how = 'any', subset = 'score')
df[['home_score', 'away_score']] = df['score'].str.split(':', expand=True)
df['home_score'] = pd.to_numeric(df['home_score'])
df['away_score'] = pd.to_numeric(df['away_score'])
df = df[(~df['home_score'].isnull()) | (~df['away_score'].isnull())]

df['result'] = df.apply(lambda row: '1' if row['home_score'] > row['away_score']
                                        else '2' if row['home_score'] < row['away_score']
                                        else 'X', axis=1)



-goals scores (at home and away)  
-goals conceded (at home and away)  
-goal difference  
-current ranking  
-average ranking in the last 5 seasons for example  
-current streak of games (average number of points gained in the last X games)  
-head2head (goals scored, conceded, wins, ties...)  
-general win/tie/loses rate of each team  
-Number of promotions and relegations ?  
-biggest win ?  
-longest undefeated streak ?  
-day and hour ?  

In [41]:
df['goal_difference'] = (df['home_score'] - df['away_score']).abs()

df['last_season_ranking'] = df.groupby(['season', 'home_team'])['home_score'].shift(1).rank(method='dense', ascending=False)

print(df.head())

      season  division  matchday     date  time    home_team        away_team  \
0  1928-1929         1         1  2/10/29  None  Arenas Club  Athletic Madrid   
1  1928-1929         1         1  2/10/29  None     Espanyol       Real Unión   
2  1928-1929         1         1  2/10/29  None  Real Madrid        Catalunya   
3  1928-1929         1         1  2/10/29  None     Donostia         Athletic   
4  1928-1929         1         1  2/12/29  None       Racing        Barcelona   

  score  home_score  away_score result  goal_difference  live_ranking  \
0   2:3           2           3      2                1             2   
1   3:2           3           2      1                1             2   
2   5:0           5           0      1                5             1   
3   1:1           1           1      X                0             4   
4   0:2           0           2      2                2             5   

   last_season_ranking  
0                  NaN  
1                  NaN  